In [2]:
import pandas as pd

DATA_PATH = '../../../data/user_ratings.csv'
full_df = pd.read_csv(DATA_PATH)

small_df = full_df.sample(1000, random_state=42)
small_df

,BGGId,Rating,Username
16597337,17709,2.0,kerbythepurplecow
15421575,155693,2.0,Asgren
16051321,40531,7.0,dzudz
16392352,22864,7.0,Gamezombiac
5484430,50,8.0,jaya
...,...,...,...
9028411,164153,9.0,Treble Delta
5500626,144733,7.6,hisayasu
17303655,11104,5.0,apekoolp
11167086,192458,8.0,DuneTiger


# Most popular items
- compute average rating for each item
- recommend items with the highest averages

In [4]:
average_ratings = small_df[['BGGId', 'Rating']].groupby('BGGId').agg(['mean', 'count']).sort_values(by=('Rating', 'mean'), ascending=False)
average_ratings.head(10)

Rating      
         mean count
BGGId              
19370    10.0     1
248562   10.0     1
35423    10.0     1
219217   10.0     1
4921     10.0     1
233078   10.0     1
4112     10.0     1
163166   10.0     1
151070   10.0     1
216132   10.0     1

## Problems:
### Number of ratings, uncertainty
- average 5 from 3 ratings
- average 4.9 from 100 ratings

In [5]:
MIN_RATINGS = 3
average_ratings[average_ratings[('Rating', 'count')] > MIN_RATINGS].head(10)

Rating      
         mean count
BGGId              
237182  8.625     4
96848   8.625     4
2651    8.625     4
169786  8.375     4
133473  8.315     4
84876   8.300     5
170216  8.100     5
178900  8.050     4
68448   7.800     5
201808  7.750     4

### Bias, normalization
- some users give systematically higher ratings

In [17]:
user_means = full_df[['Rating', 'Username']].groupby(['Username']).agg(['mean', 'count']).sort_values(by=('Rating', 'count'), ascending=False)
user_means

Rating      
                     mean count
Username                       
oldgoat3769967   6.202526  6493
warta            7.229550  6247
leffe dubbel     5.952538  6047
TomVasel         6.417876  5706
Doel             7.508228  5226
...                   ...   ...
Knooks          10.000000     1
Torias          10.000000     1
Torian80        10.000000     1
johndicksa      10.000000     1
MrBagoo          8.000000     1

[411374 rows x 2 columns]

In [32]:
small_df['UnbiasedRating'] = small_df['Rating'] - user_means.loc[small_df['Username']]['Rating']['mean'].values
small_df

,BGGId,Rating,Username,UnbiasedRating
16597337,17709,2.0,kerbythepurplecow,-3.967290
15421575,155693,2.0,Asgren,-5.226042
16051321,40531,7.0,dzudz,0.209220
16392352,22864,7.0,Gamezombiac,0.319951
5484430,50,8.0,jaya,1.306312
...,...,...,...,...
9028411,164153,9.0,Treble Delta,1.642857
5500626,144733,7.6,hisayasu,-0.027440
17303655,11104,5.0,apekoolp,-1.413043
11167086,192458,8.0,DuneTiger,1.298068


In [36]:
average_unbiased_ratings = small_df[['BGGId', 'UnbiasedRating']].groupby('BGGId').agg(['mean', 'count']).sort_values(by=('UnbiasedRating', 'mean'), ascending=False)
average_unbiased_ratings[average_unbiased_ratings[('UnbiasedRating', 'count')] > MIN_RATINGS].head(10)

UnbiasedRating      
                 mean count
BGGId                      
2651         1.701470     4
170216       1.305632     5
68448        1.016412     5
169786       0.987679     4
96848        0.906681     4
237182       0.772956     4
133473       0.752472     4
84876        0.663215     5
201808       0.608244     4
178900       0.467520     4

## Exploitation vs Exploration
- "pure exploitation" – always recommend "top items" – what we did above
- what if some other item is actually better, rating is poorer just due to noise?
- "exploration" – presenting items to get more data
- Multi-armed Bandit
  - standard model for "exploitation vs exploration"
  - many algorithms (e.g., "upper confidence bounds")
- core idea:
  - do not use just "averages"
  - quantify uncertainty (e.g., standard deviation)
  - systematic approach: Bayesian statistics
  - pragmatic approach: U(n) ∼ 1/n, roulette wheel selection
